In [1]:
print("OK")

OK


In [39]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers

In [52]:
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
HF_TOKEN = os.getenv('HF_TOKEN')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [8]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob = "*.pdf", loader_cls = PyPDFLoader)

    document = loader.load()
    return document

In [12]:
extracted_data = load_pdf("data/")

### Create text chunks

In [14]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)

### Embedding text chunks

In [18]:
def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [19]:
embedding = download_hf_embeddings()

C:\Users\Karthik\AppData\Local\Temp\ipykernel_25948\2096708754.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Karthik\OneDrive\Desktop\medical chatbot\env\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Karthik\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by sett

In [20]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Initialize Pinecone

In [42]:
index_name = "medical-chatbot"

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name = index_name)

In [44]:
docsearch = Pinecone.from_existing_index(index_name, embedding)

query = "What are allergies?"

docs = docsearch.similarity_search(query, k = 3)

print(docs)

[Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.eyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptomsinclude rash, inflammation, sneezing, itchy wateryeyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.'), Document(page_content='Although environmental medicine is gaining more\nrespect within conventional medicine, detoxificationKEY TERMS\nAllergen —A foreign substance, such as mites in\nhouse dust or animal dander, that wheninhaled,causes the airways to narrow and pro-duces symptoms of asthma.\nAntibody —A protein, also called immunoglobu-\nlin, produced by immune system cells to removeantigens (the foreign substances that trigger theimmune response).\nFibromyalgia —A condition of debilitating pain,

In [45]:
prompt_template = """
Use the following pieces of information to answer the user's question
If you don't know the answer just say you don't know. Do not try to make up answers.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else:
Helpful answer: 
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables = ["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

### Loading the llama 3.1 model

In [57]:
from langchain.llms import HuggingFaceHub


llm = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3.1-8B", huggingfacehub_api_token=HF_TOKEN)

In [59]:
llm("What is the capital of France?")

HfHubHTTPError:  (Request ID: SHwFv_0xnOC37ixLyPYO3)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3.1-8B.
If you are trying to create or update content, make sure you have a token with the `write` role.
The model meta-llama/Meta-Llama-3.1-8B is too large to be loaded automatically (16GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).